# Tools and Routing

**Takeaways**
- Differnet ways of converting objects into OpenAI functions
    - Pydantic object: convert_pydantic_to_openai_function(Class)
    - Functions (with Pydantic schema): format_tool_to_openai_function
    - OpenApi_spec: openapi_spec_to_openai_fn
- Without Pydantic schema, the function will be called with the arguments as a dictionary (e.g. search_wikipedia({"query": "langchain"}))
- Use OpenAIFunctionsAgentOutputParser to route between an a function call and str output, which can be printed out using a "route" helper function
- Use "from langchain_openai import ChatOpenAI" to replace "from langchain.chat_models import ChatOpenAI"


In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [3]:
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [4]:
search.name, search.description, search.args

('search',
 'Search for weather online',
 {'query': {'title': 'Query', 'type': 'string'}})

In [5]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [6]:
# Once a tool decorator is added, the function becomes a Langchain tool object
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [7]:
search.run("sf")
search("sf")

/var/folders/dp/j76v96ld7kgfdxz859wprxkw0000gn/T/ipykernel_46420/2822380983.py:2: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  search("sf")


'42f'

In [8]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    # ... means latitude is required
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        print(results)
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [9]:
get_current_temperature.name, get_current_temperature.description, get_current_temperature.args

('get_current_temperature',
 'Fetch current temperature for given coordinates.',
 {'latitude': {'description': 'Latitude of the location to fetch weather data for',
   'title': 'Latitude',
   'type': 'number'},
  'longitude': {'description': 'Longitude of the location to fetch weather data for',
   'title': 'Longitude',
   'type': 'number'}})

In [10]:
from langchain.tools.render import format_tool_to_openai_function

In [11]:
format_tool_to_openai_function(get_current_temperature)

/var/folders/dp/j76v96ld7kgfdxz859wprxkw0000gn/T/ipykernel_46420/540789164.py:1: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [12]:
get_current_temperature({"latitude": 13, "longitude": 14})

{'latitude': 13.0, 'longitude': 14.0, 'generationtime_ms': 0.019669532775878906, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 280.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-08-09T00:00', '2025-08-09T01:00', '2025-08-09T02:00', '2025-08-09T03:00', '2025-08-09T04:00', '2025-08-09T05:00', '2025-08-09T06:00', '2025-08-09T07:00', '2025-08-09T08:00', '2025-08-09T09:00', '2025-08-09T10:00', '2025-08-09T11:00', '2025-08-09T12:00', '2025-08-09T13:00', '2025-08-09T14:00', '2025-08-09T15:00', '2025-08-09T16:00', '2025-08-09T17:00', '2025-08-09T18:00', '2025-08-09T19:00', '2025-08-09T20:00', '2025-08-09T21:00', '2025-08-09T22:00', '2025-08-09T23:00'], 'temperature_2m': [26.8, 26.4, 26.0, 25.7, 25.4, 25.3, 25.6, 26.4, 27.8, 29.2, 30.5, 31.7, 32.4, 33.0, 33.0, 32.7, 31.8, 30.5, 29.6, 29.0, 28.1, 27.4, 27.0, 26.8]}}


'The current temperature is 26.4°C'

In [13]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [14]:
search_wikipedia.name, search_wikipedia.description, 

('search_wikipedia', 'Run Wikipedia search and get page summaries.')

In [15]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [16]:
print(search_wikipedia({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Vector database
Summary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more approximate nearest neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.
Vectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending on the c

In [28]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
# from langchain.utilities.openapi import OpenAPISpec
from langchain_community.utilities.openapi import OpenAPISpec

In [25]:
# For conversion into LangChain-compatible functions using openapi_spec_to_openai_fn()
# Openapi_spec

text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [31]:
text = '''
openapi: 3.0.0
info:
  version: 1.0.0
  title: Swagger Petstore
  license:
    name: MIT
servers:
  - url: http://petstore.swagger.io/v1
paths:
  /pets:
    get:
      summary: List all pets
      operationId: listPets
      tags:
        - pets
      parameters:
        - name: limit
          in: query
          description: How many items to return at one time (max 100)
          required: false
          schema:
            type: integer
            maximum: 100
            format: int32
      responses:
        '200':
          description: A paged array of pets
          headers:
            x-next:
              description: A link to the next page of responses
              schema:
                type: string
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Pets'
        default:
          description: unexpected error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Error'
    post:
      summary: Create a pet
      operationId: createPets
      tags:
        - pets
      responses:
        '201':
          description: Null response
        default:
          description: unexpected error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Error'
  /pets/{petId}:
    get:
      summary: Info for a specific pet
      operationId: showPetById
      tags:
        - pets
      parameters:
        - name: petId
          in: path
          required: true
          description: The id of the pet to retrieve
          schema:
            type: string
      responses:
        '200':
          description: Expected response to a valid request
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Pet'
        default:
          description: unexpected error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Error'
components:
  schemas:
    Pet:
      type: object
      required:
        - id
        - name
      properties:
        id:
          type: integer
          format: int64
        name:
          type: string
        tag:
          type: string
    Pets:
      type: array
      maxItems: 100
      items:
        $ref: '#/components/schemas/Pet'
    Error:
      type: object
      required:
        - code
        - message
      properties:
        code:
          type: integer
          format: int32
        message:
          type: string
'''

In [33]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


AttributeError: 'super' object has no attribute 'parse_obj'

In [37]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [38]:
pet_openai_functions, pet_callables

([{'name': 'listPets',
   'description': 'List all pets',
   'parameters': {'type': 'object',
    'properties': {'params': {'type': 'object',
      'properties': {'limit': {'type': 'integer',
        'maximum': 100.0,
        'schema_format': 'int32',
        'description': 'How many items to return at one time (max 100)'}},
      'required': []}}}},
  {'name': 'createPets',
   'description': 'Create a pet',
   'parameters': {'type': 'object', 'properties': {}}},
  {'name': 'showPetById',
   'description': 'Info for a specific pet',
   'parameters': {'type': 'object',
    'properties': {'path_params': {'type': 'object',
      'properties': {'petId': {'type': 'string',
        'description': 'The id of the pet to retrieve'}},
      'required': ['petId']}}}}],
 <function langchain.chains.openai_functions.openapi.openapi_spec_to_openai_fn.<locals>.default_call_api(name: 'str', fn_args: 'dict', headers: 'Optional[dict]' = None, params: 'Optional[dict]' = None, **kwargs: 'Any') -> 'Any'>)

In [34]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
resp = model.invoke("tell me about pet with id 42")
resp

AIMessage(content="I'm sorry, but I don't have access to specific databases or records to provide information about a pet with a specific ID. If you have a specific context or details about the pet, I would be happy to help with general information or advice related to pets!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 15, 'total_tokens': 66, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C2Sc21x01tHrl8AapHTu0rfWJkesh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d0640089-e2a4-4bd8-bb79-3888a7dd1560-0', usage_metadata={'input_tokens': 15, 'output_tokens': 51, 'total_tokens': 66, 'input_token_details': {'audio': 0, 'cache_read': 0},

In [38]:
# from langchain.chat_models import ChatOpenAI

In [39]:
model = ChatOpenAI(temperature=0, model="gpt-4o-mini").bind(functions=pet_openai_functions)

NameError: name 'pet_openai_functions' is not defined

In [40]:
model.invoke("what are three pets names")

AIMessage(content='Here are three popular pet names:\n\n1. Bella\n2. Max\n3. Charlie\n\nThese names are commonly used for dogs and cats!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 12, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C2ScIpPZzMrf0WPdoiO7K5arE3bL9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9c205ce1-f39e-4bf1-b0c4-b1e188631306-0', usage_metadata={'input_tokens': 12, 'output_tokens': 29, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [41]:
model.invoke("tell me about pet with id 42")

AIMessage(content="I'm sorry, but I don't have access to specific databases or records to look up information about a pet with ID 42. If you provide more context or details about the pet, I might be able to help you with general information or advice related to pets!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 15, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C2ScMSFyulRqKuBOpzBMN921jww1z', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--04985452-918d-42a5-8199-22e3fc78b6cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 52, 'total_tokens': 67, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output

### Routing

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [42]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [43]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 105, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--dc00ed39-19b3-4844-a8e2-70f155033621-0')

In [44]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 101, 'total_tokens': 117, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--d492a385-0efd-41e2-b8a8-f338e05b968a-0')

In [45]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])
chain = prompt | model

In [46]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 113, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run--79750150-c04b-4b68-adcb-61670537436a-0')

In [47]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [48]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [49]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [50]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [51]:
result.tool

'get_current_temperature'

In [52]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [53]:
get_current_temperature(result.tool_input)

{'latitude': 37.763283, 'longitude': -122.41286, 'generationtime_ms': 0.026464462280273438, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 18.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-08-09T00:00', '2025-08-09T01:00', '2025-08-09T02:00', '2025-08-09T03:00', '2025-08-09T04:00', '2025-08-09T05:00', '2025-08-09T06:00', '2025-08-09T07:00', '2025-08-09T08:00', '2025-08-09T09:00', '2025-08-09T10:00', '2025-08-09T11:00', '2025-08-09T12:00', '2025-08-09T13:00', '2025-08-09T14:00', '2025-08-09T15:00', '2025-08-09T16:00', '2025-08-09T17:00', '2025-08-09T18:00', '2025-08-09T19:00', '2025-08-09T20:00', '2025-08-09T21:00', '2025-08-09T22:00', '2025-08-09T23:00'], 'temperature_2m': [23.7, 21.9, 18.2, 16.3, 15.0, 14.8, 14.6, 14.3, 13.9, 13.9, 13.6, 13.3, 13.6, 13.5, 13.6, 14.5, 15.1, 16.7, 18.2, 22.5, 22.7, 22.0, 21.6, 20.9]}}


'The current temperature is 21.9°C'

In [54]:
result = chain.invoke({"input": "hi!"})

In [55]:
type(result)

langchain_core.agents.AgentFinish

In [56]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

In [57]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [58]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [60]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})
result

{'latitude': 37.763283, 'longitude': -122.41286, 'generationtime_ms': 0.02396106719970703, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 18.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-08-09T00:00', '2025-08-09T01:00', '2025-08-09T02:00', '2025-08-09T03:00', '2025-08-09T04:00', '2025-08-09T05:00', '2025-08-09T06:00', '2025-08-09T07:00', '2025-08-09T08:00', '2025-08-09T09:00', '2025-08-09T10:00', '2025-08-09T11:00', '2025-08-09T12:00', '2025-08-09T13:00', '2025-08-09T14:00', '2025-08-09T15:00', '2025-08-09T16:00', '2025-08-09T17:00', '2025-08-09T18:00', '2025-08-09T19:00', '2025-08-09T20:00', '2025-08-09T21:00', '2025-08-09T22:00', '2025-08-09T23:00'], 'temperature_2m': [23.7, 21.9, 18.2, 16.3, 15.0, 14.8, 14.6, 14.3, 13.9, 13.9, 13.6, 13.3, 13.6, 13.5, 13.6, 14.5, 15.1, 16.7, 18.2, 22.5, 22.7, 22.0, 21.6, 20.9]}}


'The current temperature is 18.2°C'

In [63]:
chain.invoke({"input": "What is langchain?"})

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain\'s use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Vector database\nSummary: A vector database, vector store or vector search engine is a database that uses the vector space model to store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more approximate nearest neighbor algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In this space, each dimension corresponds to a feature of the data, with the number of dimensions ranging from a few hundred to tens of thousands, depending o

In [64]:
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'